In [2]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
interactions_df = pd.read_excel('Combined_Interactions.xlsx')
events_df = pd.read_excel('Events.xlsx')
responses = pd.read_csv('(Responses) (1).csv')
user_item_matrix = interactions_df.pivot_table(index='Name', columns='Event ID', values='Interaction Score', fill_value=0)
user_similarity = cosine_similarity(user_item_matrix)
def recommend_events(user,events_df,responses):
    ##### Collaboraitve Based Filtering
    user_index = user_item_matrix.index.get_loc(user)
    similar_users = np.argsort(user_similarity[user_index])[::-1][1:]  # Exclude the user itself
    recommended_events = pd.DataFrame(columns=['Event ID', 'score'])
    interacted_events = interactions_df[interactions_df['Name'] == user]
    for similar_user in similar_users:
        for event_id in user_item_matrix.columns:
            if user_item_matrix.loc[user_item_matrix.index[similar_user], event_id] > 0 and event_id not in interacted_events['Event ID'].tolist():
                score = user_item_matrix.loc[user_item_matrix.index[similar_user], event_id]
                if event_id not in recommended_events['Event ID'].tolist():
                    recommended_events = pd.concat([recommended_events, pd.DataFrame({'Event ID': [event_id], 'score': [score]})], ignore_index=True)
                else:
                    recommended_events.loc[recommended_events['Event ID'] == event_id, 'score'] += score
    recommended_events = recommended_events.merge(events_df, on='Event ID')
    recommended_events = recommended_events.sort_values(by='score', ascending=False)
    recommended_events.drop('score',axis=1,inplace=True)
    recommended_events = recommended_events.head()
    ##### Content Based Filtering
    #interactions_df = interactions_df[interactions_df['Name'] == user_name]
    responses = responses[responses['Name'] == user]
    responses['All Interests'] = responses['All Interests'].str.split(', ')
    events_df['Category'] = events_df[['Primary Category', 'Secondary Category']].apply(lambda x: ', '.join(filter(lambda y: pd.notna(y), x)), axis=1)
    events_df['Category'] = events_df['Category'].str.split(', ')
    events_df['Interest_Match'] = False
    for index, row in responses.iterrows():
        for tag in row['All Interests']:
            events_df.loc[events_df['Category'].apply(lambda x: tag in x), 'Interest_Match'] = True
    events_df = events_df[events_df['Interest_Match']==True]
    filtered_interactions_df_1 = interacted_events[~interacted_events['Interaction Score'].isin([-1, 0])]
    merged_df = pd.merge(interacted_events, events_df[['Event ID', 'Primary Category', 'Sub Category']], on='Event ID', how='left', suffixes=('_int', '_evt'))
    merged_df_1 = pd.merge(filtered_interactions_df_1, events_df[['Event ID', 'Primary Category', 'Sub Category']], on='Event ID', how='left', suffixes=('_int', '_evt'))
    filtered_interactions_df_neg_zero = interacted_events[interacted_events['Interaction Score'].isin([-1, 0])]
    categories = merged_df.sort_values('Interaction Score', ascending=False)['Primary Category_evt'].unique()
    recommended_categories = set()
    for category in categories:
        category_events = events_df[events_df['Primary Category'] == category]
        user_subcategories = merged_df_1[merged_df_1['Primary Category_evt'] == category]['Sub Category_evt'].unique()
        for subcategory in user_subcategories:
            subcategory_events = category_events[category_events['Sub Category'] == subcategory]
            new_events = subcategory_events[~subcategory_events['Event ID'].isin(merged_df_1['Event ID']) &
                                            ~subcategory_events['Event ID'].isin(recommended_events['Event ID']) &
                                            ~subcategory_events['Event ID'].isin(filtered_interactions_df_neg_zero['Event ID'])]
            if not new_events.empty:
                recommended_categories.add(category)
                recommended_events = pd.concat([recommended_events, new_events], ignore_index=True)
    # Add unrecommended events
    unrecommended_categories = set(categories) - recommended_categories
    for ur_category in unrecommended_categories:
        category_events_1 = events_df[events_df['Primary Category'] == ur_category]
        user_subcategories_1 = merged_df[merged_df['Primary Category_evt'] == ur_category]['Sub Category_evt'].unique()
        for sc in user_subcategories_1:
            subcategory_events_1 = category_events_1[category_events_1['Sub Category'] != sc]
            ur_category_events = subcategory_events_1[
                (~subcategory_events_1['Event ID'].isin(merged_df['Event ID'])) &
                (~subcategory_events_1['Event ID'].isin(recommended_events['Event ID']))]
            recommended_events = pd.concat([recommended_events, ur_category_events], ignore_index=True)
    # Add additional events based on sorting
    sorted_events = merged_df.sort_values('Interaction Score', ascending=False)
    for index, event in sorted_events.iterrows():
        additional_events = events_df[
            (events_df['Primary Category'] == event['Primary Category_evt']) &
            (~events_df['Event ID'].isin(merged_df['Event ID'])) &
            (~events_df['Event ID'].isin(recommended_events['Event ID']))]
        recommended_events = pd.concat([recommended_events, additional_events], ignore_index=True)
    return recommended_events
user='Pratik Bhande'
recommended_events = recommend_events(user,events_df,responses)
recommended_events.head(10)

C:\Users\pratik\AppData\Local\Temp\ipykernel_5596\2279147971.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  responses['All Interests'] = responses['All Interests'].str.split(', ')


,Event ID,Event Names,Description,Primary Category,Secondary Category,Sub Category,Location,Prices,Date,Unnamed: 9,Time,Links,Category,Interest_Match
0,EVT0022,Fun and Fair,"Don’t miss new attractions, the thrills of the...",Music and Entertainment,Food and Drinks,Exhibition,Erandwane,0,23/3/24 to 31/3/24,23/3/24 to 31/3/24,15:00:00,NaN,NaN,NaN
1,EVT0085,Dandeli Jungle Tour,उनाड भ्रमंती is happy to invite you for *Dande...,Adventure,Travel,Hiking and Trekking,Sangamvadi,10000,2024-03-29 00:00:00,Friday,00:00:00,https://allevents.in/pune/dandeli-jungle-tour-...,NaN,NaN
2,EVT0030,Matheran Forest Camping,NaN,Travel,NaN,Camping,Sangamvadi,1499,2024-02-06 00:00:00,Tuesday,19:00:00,NaN,NaN,NaN
3,EVT0147,Flea Market - local shopping,Explore innovative startups and entrepreneurs ...,Music and Entertainment,Food and Drinks,Shopping,Karve Nagar,50,2024-04-13 00:00:00,NaN,17:00:00,NaN,NaN,NaN
4,EVT0025,Shopping Festival,NaN,Music and Entertainment,Food and Drinks,Exhibition,Magarpatta,0,23/3/24 to 31/3/24,23/3/24 to 31/3/24,11:00:00,NaN,NaN,NaN
5,EVT0081,Kolad River Side Camping and Water Sports,*Dear people* Adventure Camp Zone ( Kolad )Get...,Adventure,Travel,Adventure Sports,Deccan,1500,2024-03-31 00:00:00,Sunday,15:00:00,https://allevents.in/roha/kolad-river-side-cam...,"[Adventure, Travel]",True
6,EVT0131,Ambenali Climbing Challenge,Join us for one of the simplest and safest cli...,Adventure,NaN,Adventure Sports,Shivajinagar,1800,2024-04-14 00:00:00,NaN,06:00:00,https://allevents.in/mahabaleswar/ambenali-cli...,[Adventure],True
7,EVT0135,2024 Gentleman's Ride,"Register to join an event like no other, as we...",Adventure,NaN,Adventure Sports,Parvati,1100,2024-05-19 00:00:00,NaN,04:30:00,NaN,[Adventure],True
8,EVT0203,Mountain Zipline at Malhar Matchi,We are ready to fly without wings and we want ...,Adventure,NaN,Adventure Sports,Shivajinagar,NaN,Multiple Dates,NaN,09:00:00,in.bookmyshow.com/activities/flying-baba-mount...,[Adventure],True
9,EVT0132,Climb Koleshwar Plateau,"Pitch your tents, Dinner and Stay near Jambhal...",Adventure,NaN,Hiking and Trekking,Sangamvadi,2000,2024-04-30 00:00:00,NaN,06:00:00,NaN,[Adventure],True


In [4]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

interactions_df = pd.read_excel('Combined_Interactions.xlsx')
events_df = pd.read_excel('Events.xlsx')
responses = pd.read_csv('(Responses) (1).csv')

user_item_matrix = interactions_df.pivot_table(index='Name', columns='Event ID', values='Interaction Score', fill_value=0)

user_similarity = cosine_similarity(user_item_matrix)

def recommend_events(user,events_df,responses):

    ##### Collaboraitve Based Filtering

    user_index = user_item_matrix.index.get_loc(user)
    similar_users = np.argsort(user_similarity[user_index])[::-1][1:] 
    similar_users = similar_users[:5]# Exclude the user itself

    recommended_events = pd.DataFrame(columns=['Event ID', 'score'])

    interacted_events = interactions_df[interactions_df['Name'] == user]

    for similar_user in similar_users:
        for event_id in user_item_matrix.columns:
            if user_item_matrix.loc[user_item_matrix.index[similar_user], event_id] > 0 and event_id not in interacted_events['Event ID'].tolist():
                score = user_item_matrix.loc[user_item_matrix.index[similar_user], event_id]
                if event_id not in recommended_events['Event ID'].tolist():
                    recommended_events = pd.concat([recommended_events, pd.DataFrame({'Event ID': [event_id], 'score': [score]})], ignore_index=True)
                else:
                    recommended_events.loc[recommended_events['Event ID'] == event_id, 'score'] += score

    recommended_events = recommended_events.merge(events_df, on='Event ID')
    recommended_events = recommended_events.sort_values(by='score', ascending=False)
    recommended_events.drop('score',axis=1,inplace=True)
    recommended_events=recommended_events.head()

    ##### Content Based Filtering

    #interactions_df = interactions_df[interactions_df['Name'] == user_name]
    responses = responses[responses['Name'] == user]
    responses['All Interests'] = responses['All Interests'].str.split(', ')
    events_df['Category'] = events_df[['Primary Category', 'Secondary Category']].apply(lambda x: ', '.join(filter(lambda y: pd.notna(y), x)), axis=1)
    events_df['Category'] = events_df['Category'].str.split(', ')
    events_df['Interest_Match'] = False
    for index, row in responses.iterrows():
        for tag in row['All Interests']:
            events_df.loc[events_df['Category'].apply(lambda x: tag in x), 'Interest_Match'] = True
    events_df = events_df[events_df['Interest_Match']==True]

    filtered_interactions_df_1 = interacted_events[~interacted_events['Interaction Score'].isin([-1, 0])]
    merged_df = pd.merge(interacted_events, events_df[['Event ID', 'Primary Category', 'Sub Category']], on='Event ID', how='left', suffixes=('_int', '_evt'))
    merged_df_1 = pd.merge(filtered_interactions_df_1, events_df[['Event ID', 'Primary Category', 'Sub Category']], on='Event ID', how='left', suffixes=('_int', '_evt'))
    filtered_interactions_df_neg_zero = interacted_events[interacted_events['Interaction Score'].isin([-1, 0])]


    categories = merged_df.sort_values('Interaction Score', ascending=False)['Primary Category_evt'].unique()
    recommended_categories = set()

    for category in categories:
        category_events = events_df[events_df['Primary Category'] == category]
        user_subcategories = merged_df_1[merged_df_1['Primary Category_evt'] == category]['Sub Category_evt'].unique()
        for subcategory in user_subcategories:
            subcategory_events = category_events[category_events['Sub Category'] == subcategory]
            new_events = subcategory_events[~subcategory_events['Event ID'].isin(merged_df_1['Event ID']) &
                                            ~subcategory_events['Event ID'].isin(recommended_events['Event ID']) &
                                            ~subcategory_events['Event ID'].isin(filtered_interactions_df_neg_zero['Event ID'])]
            if not new_events.empty:
                recommended_categories.add(category)
                recommended_events = pd.concat([recommended_events, new_events], ignore_index=True)
    # Add unrecommended events
    unrecommended_categories = set(categories) - recommended_categories
    for ur_category in unrecommended_categories:
        category_events_1 = events_df[events_df['Primary Category'] == ur_category]
        user_subcategories_1 = merged_df[merged_df['Primary Category_evt'] == ur_category]['Sub Category_evt'].unique()
        for sc in user_subcategories_1:
            subcategory_events_1 = category_events_1[category_events_1['Sub Category'] != sc]
            ur_category_events = subcategory_events_1[
                (~subcategory_events_1['Event ID'].isin(merged_df['Event ID'])) &
                (~subcategory_events_1['Event ID'].isin(recommended_events['Event ID']))]
            recommended_events = pd.concat([recommended_events, ur_category_events], ignore_index=True)
    # Add additional events based on sorting
    sorted_events = merged_df.sort_values('Interaction Score', ascending=False)
    for index, event in sorted_events.iterrows():
        additional_events = events_df[
            (events_df['Primary Category'] == event['Primary Category_evt']) &
            (~events_df['Event ID'].isin(merged_df['Event ID'])) &
            (~events_df['Event ID'].isin(recommended_events['Event ID']))]
        recommended_events = pd.concat([recommended_events, additional_events], ignore_index=True)

    return recommended_events


user='Pratik Bhande'
recommended_events = recommend_events(user,events_df,responses)

recommended_events.head(10)

C:\Users\pratik\AppData\Local\Temp\ipykernel_5596\1656568444.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  responses['All Interests'] = responses['All Interests'].str.split(', ')


,Event ID,Event Names,Description,Primary Category,Secondary Category,Sub Category,Location,Prices,Date,Unnamed: 9,Time,Links,Category,Interest_Match
0,EVT0030,Matheran Forest Camping,NaN,Travel,NaN,Camping,Sangamvadi,1499,2024-02-06 00:00:00,Tuesday,19:00:00,NaN,NaN,NaN
1,EVT0085,Dandeli Jungle Tour,उनाड भ्रमंती is happy to invite you for *Dande...,Adventure,Travel,Hiking and Trekking,Sangamvadi,10000,2024-03-29 00:00:00,Friday,00:00:00,https://allevents.in/pune/dandeli-jungle-tour-...,NaN,NaN
2,EVT0147,Flea Market - local shopping,Explore innovative startups and entrepreneurs ...,Music and Entertainment,Food and Drinks,Shopping,Karve Nagar,50,2024-04-13 00:00:00,NaN,17:00:00,NaN,NaN,NaN
3,EVT0009,Ensemble - Fashion and Lifestyle Exibition,A carefully curated showcase of some wonderful...,Fashion,NaN,Exhibition,Camp,0,2024-03-08 00:00:00,Friday,11:00:00,https://allevents.in/pune/ensemble-a-fashion-a...,NaN,NaN
4,EVT0129,Business Shower - share your story,This is a stage where founders get a chance to...,Business and Networking,NaN,Seminar,Pune,Free,2024-03-30 00:00:00,NaN,17:00:00,NaN,NaN,NaN
5,EVT0081,Kolad River Side Camping and Water Sports,*Dear people* Adventure Camp Zone ( Kolad )Get...,Adventure,Travel,Adventure Sports,Deccan,1500,2024-03-31 00:00:00,Sunday,15:00:00,https://allevents.in/roha/kolad-river-side-cam...,"[Adventure, Travel]",True
6,EVT0131,Ambenali Climbing Challenge,Join us for one of the simplest and safest cli...,Adventure,NaN,Adventure Sports,Shivajinagar,1800,2024-04-14 00:00:00,NaN,06:00:00,https://allevents.in/mahabaleswar/ambenali-cli...,[Adventure],True
7,EVT0135,2024 Gentleman's Ride,"Register to join an event like no other, as we...",Adventure,NaN,Adventure Sports,Parvati,1100,2024-05-19 00:00:00,NaN,04:30:00,NaN,[Adventure],True
8,EVT0203,Mountain Zipline at Malhar Matchi,We are ready to fly without wings and we want ...,Adventure,NaN,Adventure Sports,Shivajinagar,NaN,Multiple Dates,NaN,09:00:00,in.bookmyshow.com/activities/flying-baba-mount...,[Adventure],True
9,EVT0132,Climb Koleshwar Plateau,"Pitch your tents, Dinner and Stay near Jambhal...",Adventure,NaN,Hiking and Trekking,Sangamvadi,2000,2024-04-30 00:00:00,NaN,06:00:00,NaN,[Adventure],True


### Code Flow and Explanation:

1. **Import Libraries**:
   - `pandas` for data manipulation.
   - `cosine_similarity` from `sklearn.metrics.pairwise` for similarity computation.
   - `numpy` for numerical operations.

2. **Load Data**:
   - Load interaction data (`Combined_Interactions.xlsx`), event details (`Events.xlsx`), and user responses (`(Responses) (1).csv`) into dataframes.

3. **User-Item Matrix Creation**:
   - Pivot the interaction data to create a **user-item matrix** where rows are users, columns are event IDs, and values are interaction scores. Missing values are filled with 0.

4. **Compute User Similarity**:
   - Use cosine similarity to calculate the **user-user similarity matrix** based on their event interactions.

5. **Collaborative Filtering for Recommendations**:
   - The `recommend_events()` function performs **collaborative filtering**:
     - Retrieve the most similar users to the target user, excluding the user itself.
     - Recommend events that similar users have interacted with, but the target user has not.
     - Store recommended events along with the similarity scores from the similar users.
     - Merge recommendations with event details from `events_df`, sorting by the highest scores.

6. **Content-Based Filtering for Recommendations**:
   - Use user interests from the `responses` dataframe to perform **content-based filtering**:
     - Match the user's interests to event categories and flag matching events.
     - Merge the user's past interactions with the matching events, focusing on events that haven't been interacted with.
     - Recommend new events within the same categories and subcategories as the user's prior interactions.

7. **Handle Unrecommended Events**:
   - If certain categories haven't been recommended yet, additional events from these categories are included.
   - This ensures diversity in the recommendations by introducing events from categories the user has not interacted with.

8. **Final Event Selection**:
   - Additional events are recommended based on sorting by interaction score.
   - The final set of recommendations is returned.

9. **Example Usage**:
   - The function is called for the user `Pratik Bhande`, generating the top 10 event recommendations.